# Computational grid for the Richards' equation
----
**Author:** [Niccolò Tubini](https://www.researchgate.net/profile/Niccolo_Tubini2), [Riccardo Rigon](https://scholar.google.it/citations?user=OB1jx0IAAAAJ&hl=it)
 

**Documentation Author:** [Niccolò Tubini](https://www.researchgate.net/profile/Niccolo_Tubini2)

**To whom address questions:** 
 - [Niccolò Tubini](https://www.researchgate.net/profile/Niccolo_Tubini2) 
 - [GEOframe Users Group](https://groups.google.com/forum/#!forum/geoframe-components-users)
 - [GEOframe Developers Mailing List](https://groups.google.com/forum/#!forum/geoframe-components-developers)
 
**Version:** 0.98

**Keywords:** OMS3, WHETGEO-1D, Richards 

**License:** [GPL3 v3](https://www.gnu.org/licenses/gpl-3.0.en.html)

## Table of Contents

* [Abstract](#Abstract)

* [Grid input files](#Grid-input-files)
  * [The geometry](#The-geometry)
      * [Example of grid file](#Example-of-grid-file)
  * [The initial condition](#The-initial-condition)
      * [Example of initial condition file](#Example-of-the-initial-condition-file)
  * [The parameters](#The-parameters)
      * [Example of parameter files](#Example-of-parameter-files)
* [Notebook](#Notebook)

* [Grid output file](#Grid-output-file)

* [Example of domain discretization](#Example-of-domain-discretization)


# Abstract

Solving a partial differential equation (PDE) with a numerical method requires the definition of the computational grid. The computational grid includes:
- the discretization of the domain. The geometric discretization of the physical domain results in a mesh on which the conservation equations are eventually solved. This requires the subdivision of the domain into discrete non-overlapping cells or elements that completelyfill the computational domain to yield a grid or mesh system ([1][ref1])
- the definition of the parameters and of the equations state
- the definition of the initial condition

[ref1]: https://www.springer.com/gp/book/9783319168739
   
   
# Grid input files

The input files to define the computational grid are three:
- one defining the geometry 
- one defining the initial condition
- one defining the parameters

All these files are stored in `/data/Grid_input`


## The geometry

This is an example of the input file for the grid to simulate di Richards' equation

|Type      |eta    |K      |equationStateID     |parameterID    |
|----------|-------|-------|--------------------|---------------|
|L         |0      |150    |0                   |1              |
|L         |-1     |100    |0                   |2              |
|L         |-2     |25     |0                   |3              |
|L         |-3     |nan    |nan                 |nan            |


**Type**  if L defines the interface separating two different layers. The first and the line must be L: these lines defines the borders of the domain. M can be used to insert a measurament point and it is required when you want to calibrate/validate the model against some measuraments.

**eta** [m] is the vertical coordinate positive upward with the origin set at the soil surface.

**K** is the number of control volume to discretize each layer of your domain. As you increase the spatial resolution, larger K value, you better approximate the solution of the PDE. This is not for free since also the computational time increases and the space on the disk to save the output. A good spatial resolution can be $0.005~0.01$ [m]. 
<figure>
    <img src="Figures/grid_classical.PNG" width="400" height="400/1.618">
    <figcaption>Fig.1 - Example of a domain discretization for the Richards' equation. This discretization is `classical` with control volume size constant in each soil layer. </figcaption>
</figure>
In the notebook it is necessary to specify
    -`grid_type` = 'classical'

This is not the only possible subvidision. The domain can be discretized in other two manner. 
Figure (2) shows an example of a domain subdivision obtained with the exponential function ([1](https://gmd.copernicus.org/articles/6/1319/2013/))
$$dz_i = dz_{min}(1+b)(i-1)$$
. Here the domain is not discretized by using the values K of the input file but specifying in the notebook
    -`grid_type` = 'exponential'

    -`dz_min` = the size of the uppermost control volume 

    -`dz_max` = the maximum size of the control volume

    -`b` = is the growth rate 
<figure>
    <img src="Figures/grid_exponential.PNG" width="400" height="400/1.618">
    <figcaption>Fig.1 - Example of a domain discretization for the Richards' equation. This discretization is `exponential`: the control volume size varies exponentially with depth. In this case you have a finer grid at top of the domain, where you observe the stronger gradient of $\psi$, that becomes coarser as you move downward. This choice can be convinient when you are interested to capture the spatial variability in the uppermost part of the domain and at the same time do not want to have too many computational points, aka save time and space on the disk. </figcaption>
</figure>

The last option is a combination of the previous two, Fig. (3).
<figure>
    <img src="Figures/grid_mixed.PNG" width="400" height="400/1.618">
    <figcaption>Fig.3 - Example of a domain discretization for the Richards' equation. This discretization is `mixed`: close to the interface the control volume size varies exponentially with depth and cannot exceed a prescribed size. In this case the discretization is more homogeneous compared to the exponential grid and it is finer at the interfaces of the soil layers where the abrput change in the soil parameters determines strong gradients.</figcaption>
</figure>
Here the domain is not discretized by using the values K of the input file but specifying in the notebook
    -`grid_type` = 'mixed'

    -`dz_min` = the size of the uppermost control volume 

    -`dz_max` = the maximum size of the control volume

    -`b` = is the growth rate 


**equationStateID** is an interger and identifies the equationState to use in a layer of our domain. This interger is related to the field `solver.typeClosureEquation` we have in the .sim file. If you want to solve
- only the Richards: 
   - equationStateID is 0
   - "solver.typeClosureEquation" "{Van Genuchten}"
   - "solver.typeEquationState" "{Van Genuchten}"
   - "solver.typeUHCModel" "{Mualem Van Genuchten}"
   
- the Richards' equation coupled with the shallow water to properly define the surface boundary condition
   - equationStateID is 1
   - "solver.typeClosureEquation" "{Water Depth, Van Genuchten}"
   - "solver.typeEquationState" "{Water Depth, Van Genuchten}"
   - "solver.typeUHCModel" "{null, Mualem Van Genuchten}"
In fact, as shown in Fig. (4), at the soil surface we have an additional control volume that represents the water depth at the soil surface
<figure>
    <img src="Figures/grid_coupled_mixed.PNG" width="400" height="400/1.618">
    <figcaption>Fig.4 - Example of a domain discretization for the Richards' equation coupled with the shallow water to properly define the surface boundary condition. In this case within the soil column we use one of the available SWRC models whereas the computational point at the soil surface represent the water depth for which we have a different closure equation and equation state. In the code the behaviuor of the each computational point is controlled by the equationStateID that specify wheter to use the SWRC or the water depth.</figcaption>
</figure>

**parameterID** is an integer that defines the parameter set to use in each layer of the domain. The value start from 1 and refers to a line in the parameter file.


### Example of grid file
- `data/Grid_input/_grid_Richards.csv` to simulate only the Richards equation
- `data/Grid_input/_grid_Richards_coupled.csv` to simulate the Richards' equatiion coupled with the shallow water to properly define the surface boundary condition


## The initial condition

The file is a .csv file structured in this manner

|eta   |Psi0  |T0    |
|------|------|------|
|-0    |-3.0  |273.15|
|-3    |0.0   |273.15|

**eta** [m] is the vertical coordinate positive upward with the origin set at the soil surface

**Psi0** [m] is the water suction at a different positions (eta)

**T0** [K] is the soil temperature at a different positions (eta)

From this file the profile for water suction and temperature are interpolated. The interpolation is performed by using the [scipy.interpolate.interp1d](https://docs.scipy.org/doc/scipy/reference/generated/scipy.interpolate.interp1d.html), for more details on the options refers to the documentation.



### Example of the initial condition file
- `data/Grid_input/_ic.csv`


## The parameters
The parameters file is a .csv file. For each soil water retention curve (SWRC) we have a specific file.
Here we report the header for each SWRC model:

- Van Genuchten SWRC
    \begin{equation}
        \theta(\psi) = 
        \begin{cases}
            \theta_r + (\theta_s-\theta_r)[1+(|\alpha \psi|)^{n}]^{-m} & \text{if } \psi<0 \\
            \theta_s + \rho g(\alpha_{ss}+\theta_s\beta_{ss})\psi & \text{otherwise }
        \end{cases}
    \end{equation}



 
 
 | Symbol                 | Physical quantity                   | Unit        | Input               |
 |------------------------|-------------------------------------|-------------|---------------------| 
 | $\theta_r$             |residual water content               | [-]         |thetaR               |
 | $\theta_s$             |water content at saturation          | [-]         |thetaS               |
 | $\alpha$               |Van Genuchten parameter              | [m$^{-1}$]  |alpha                |
 | $n$                    |Van Genuchten parameter ($>1$)       | [-]         |n                    |
 | $m=1-\frac{1}{n}$      |Van Genuchten parameter              | [-]         |not required         |
 | $\alpha_{ss}$          |matrix compressibility               | []          |alphaSpecificStorage |
 | $\beta{ss}$            |water compressibility                | []          |betaSpecificStorage  |
 | $K_s$                  |saturated hydraulic conductivity     | [m s$^{-1}$]|Ks                   |

    
    
    File header

   |thetaS |thetaR |n |alpha |alphaSpecificStorage |betaSpecificStorage |Ks |
   |-------|-------|--|------|---------------------|--------------------|---|
   
  
- Brooks and Corey SWRC
    \begin{equation}
        \theta(\psi) = 
        \begin{cases}
            \theta_r + (\theta_s-\theta_r)\left(\dfrac{\psi_D}{\psi}\right)^{n} & \text{if } \psi<\psi_D\\
            \theta_s + \rho g(\alpha_{ss}+\theta_s\beta_{ss})\psi & \text{otherwise }
        \end{cases}
    \end{equation}



 
 
 | Symbol                 | Physical quantity                   | Unit        | Input               |
 |------------------------|-------------------------------------|-------------|---------------------| 
 | $\theta_r$             |residual water content               | [-]         |thetaR               |
 | $\theta_s$             |water content at saturation          | [-]         |thetaS               |
 | $\psi_D$               |Brooks and Corey parameter           | [m]         |psiD                 |
 | $n$                    |Brooks and Corey parameter           | [-]         |n                    |
 | $\alpha_{ss}$          |matrix compressibility               | []          |alphaSpecificStorage |
 | $\beta{ss}$            |water compressibility                | []          |betaSpecificStorage  |
 | $K_s$                  |saturated hydraulic conductivity     | [m s$^{-1}$]|Ks                   |

    
    
    File header

   |thetaS |thetaR |n |psiD  |alphaSpecificStorage |betaSpecificStorage |Ks |
   |-------|-------|--|------|---------------------|--------------------|---|
   
- Kosugi SWRC
    \begin{equation}
        \theta(\psi) = 
        \begin{cases}
            \dfrac{\theta_s-\theta_r}{(2\pi)^{1/2}\sigma(-\psi)}\exp\left\{-\dfrac{[\ln(\psi/\psi_m)]^2}{2\sigma^2}\right\} & \text{if } \psi<\psi_D\\
            \theta_s + \rho g(\alpha_{ss}+\theta_s\beta_{ss})\psi & \text{otherwise }
        \end{cases}
    \end{equation}



 
 
 | Symbol                 | Physical quantity                   | Unit        | Input               |
 |------------------------|-------------------------------------|-------------|---------------------| 
 | $\theta_r$             |residual water content               | [-]         |thetaR               |
 | $\theta_s$             |water content at saturation          | [-]         |thetaS               |
 | $\psi_m$               |median value                         | [m]         |psiMedian            |
 | $\sigma$               |dimensionless parameter              | [-]         |sigma                |
 | $\alpha_{ss}$          |matrix compressibility               | []          |alphaSpecificStorage |
 | $\beta{ss}$            |water compressibility                | []          |betaSpecificStorage  |
 | $K_s$                  |saturated hydraulic conductivity     | [m s$^{-1}$]|Ks                   |

    
    
    File header

   |thetaS |thetaR |psiMedian |sigma    |alphaSpecificStorage |betaSpecificStorage |Ks |
   |-------|-------|----------|---------|---------------------|--------------------|---|
   
- Romano et al. SWRC
    \begin{equation}
        \theta(\psi) = 
        \begin{cases}
            \theta_r + (\theta_s-\theta_r)\left\{ w\left\{\dfrac{1}{2} \left[ \dfrac{\ln(\psi/\psi_{m1})}{\sigma_1\sqrt{2}}\right]   \right\} + (1-w)\left\{ \dfrac{1}{2} \left[ \dfrac{\ln(\psi/\psi_{m2})}{\sigma_2\sqrt{2}}\right] \right\} \right\}   & \text{if } \psi<\psi_D\\
            \theta_s + \rho g(\alpha_{ss}+\theta_s\beta_{ss})\psi & \text{otherwise }
        \end{cases}
    \end{equation}



 
 
 | Symbol                 | Physical quantity                      | Unit        | Input               |
 |------------------------|----------------------------------------|-------------|---------------------| 
 | $\theta_r$             |residual water content                  | [-]         |thetaR               |
 | $\theta_s$             |water content at saturation             | [-]         |thetaS               |
 | $w$                    |weighting factor                        | [-]         |w                    |
 | $\psi_{m1}$            |median value of water suction texture   | [m]         |h1                   |
 | $\psi_{m2}$            |median value of water suction structure | [m]         |h2                   |
 | $\sigma_{1}$           |standard deviation texture              | [-]         |sigma1               |
 | $\sigma_{2}$           |standard deviation structure            | [-]         |sigma2               |
 | $\alpha_{ss}$          |matrix compressibility                  | []          |alphaSpecificStorage |
 | $\beta{ss}$            |water compressibility                   | []          |betaSpecificStorage  |
 | $K_s$                  |saturated hydraulic conductivity        | [m s$^{-1}$]|Ks                   |

    
    
    File header

   |thetaS |thetaR |w  |sigma1  |sigma2 |h1 |h2  |alphaSpecificStorage |betaSpecificStorage |Ks |
   |-------|-------|---|--------|-------|---|----|---------------------|--------------------|---|
   
For each SWRC model it is possible to create a dataset of soil types. In the .csv file # can be used to add comments. An example is the `data/Grid_input/Richards_VG.csv`.

### Example of parameter files
- `data/Grid_input/Richards_VG.csv` for the Van Genuchten SWRC
- `data/Grid_input/Richards_BC.csv` for the Brook and Corey SWRC
- `data/Grid_input/Richards_Kosugi.csv` for the Kosugi SWRC
- `data/Grid_input/Richards_Romano.csv` for the Romano et al. SWRC



# Notebook

To elaborate the above files you can use the notebooks:

`/Jupyter_Notebook/WHETGEO1D_Richards_Computational_grid.ipynb` when you want to solve the Richards' equation considering for the surface boundary condition the Dirichlet or Neumann type.

`/Jupyter_Notebook/WHETGEO1D_RichardsCoupled_Computational_grid.ipynb` when you want to solve the Richards' equation considering for the surface boundary condition the coupling with the shallow water.


In each notebook it is reported a brief recap of the input files required to define the computational grid. About the functions used in these notebook you can find the source code in `Jupyter_Notebook/WHETGEO1D_GridCreator.py` and `Jupyter_Notebook/WHETGEO1D_toNetCDF.py`. Alternatively you can run `help(function_name)` in a cell of your notebook to read the docstring.

# Grid output file

The computational grid is stored in a netCDF file. The computational grid file are stored in the folder `/data/Grid_NetCDF`